# Capstone Week 3: Part 1 -- Ramesh Chandra Yadav

### 1. Installing necessary files and packages

In [64]:
!pip install pandas
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4

### 2. Importing necessary libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [65]:
import lxml 

### 3. Importing Postal Codes of Canada

In [24]:
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response=requests.get(url)
Toronto_neighbours=pd.read_html(response.text)

In [25]:
Toronto_neighbours

[    Postal code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 5           M6A        North York   
 6           M7A  Downtown Toronto   
 7           M8A      Not assigned   
 8           M9A         Etobicoke   
 9           M1B       Scarborough   
 10          M2B      Not assigned   
 11          M3B        North York   
 12          M4B         East York   
 13          M5B  Downtown Toronto   
 14          M6B        North York   
 15          M7B      Not assigned   
 16          M8B      Not assigned   
 17          M9B         Etobicoke   
 18          M1C       Scarborough   
 19          M2C      Not assigned   
 20          M3C        North York   
 21          M4C         East York   
 22          M5C  Downtown Toronto   
 23          M6C              York   
 24          M7C      Not assigned   
 25         

## 4. Creating Dataframe of Canada Postal codes

In [26]:
df_toronto=pd.DataFrame(Toronto_neighbours[0])
df_toronto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Postal code   180 non-null    object
 1   Borough       180 non-null    object
 2   Neighborhood  103 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


In [29]:

toronto_df=Toronto_neighbours[0]
toronto_df_neighbors=toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_df_neighbors.reset_index(drop=True, inplace=True)

x = 0
y = 0
for i in range(toronto_df_neighbors.shape[0]):
    if toronto_df_neighbors.at[i,'Neighborhood']=='Not assigned':
        toronto_df_neighbors.at[i,'Neighborhood']= toronto_df_neighborsf.at[i,'Borough']
        x=x+1
    else:    
        y=y+1
print ('{} values were changed in the Column "Neighbourhood", {} values were not'.format(x,y ))
neighbourhood_df.head()


0 values were changed in the Column "Neighbourhood", 103 values were not


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [36]:
toronto_df_neighbors.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


# Capstone Week 3: Part 2

### 1. Using previous dataframe and importing geospatial data

In [31]:
df_latlng = pd.read_csv('http://cocl.us/Geospatial_data', header=0)

In [37]:
df_latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [42]:
df_latlng.set_index('Postal Code').head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [52]:
df_latlng.rename(columns={'Postal Code':'Postal code'},inplace=True)

In [53]:
df_latlng.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [47]:
toronto_df_neighbors.set_index('Postal code').head()

,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### 2. Merging previous dataframe with geo-spatial dataframe

In [54]:
df_merge=pd.merge(toronto_df_neighbors,df_latlng)

In [61]:
df_latlng[df_latlng['Postal code']=='M5G']

,Postal code,Latitude,Longitude
57,M5G,43.657952,-79.387383


In [63]:
df_merge.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


# Capstone Week 3: part 3

In [72]:
map_clusters = folium.Map(location=[43.65, -79.4], zoom_start=11)

X = df_merge['Latitude']
Y = df_merge['Longitude']
Z = np.stack((X, Y), axis=1)

# set color scheme for the clusters
kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
df_merge['Cluster'] = clusters

# add markers to the map
for latitude, longitude, borough, cluster in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'], df_merge['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=8,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)  

map_clusters